https://www.yuque.com/ol1q37/gi94xp/gpb4iy

In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from lxml import etree
import time
import csv,os
import requests,re

In [2]:
url='https://www.baidu.com/s?wd=%E6%98%8E%E6%98%9F&rsv_spt=1&rsv_iqid=0xbe4b76860031fb66&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=&tn=baiduhome_pg&ch=&rsv_enter=1&rsv_dl=ib&inputT=2978'
driver=webdriver.Chrome()
driver.get(url)
wait = WebDriverWait(driver,10)

# 点击内地，女

In [3]:
driver.find_element_by_xpath('//*[@id="1"]/div/div[1]/div[1]/div[2]/p[1]/span[4]').click()
driver.find_element_by_xpath('//*[@id="1"]/div/div[1]/div[1]/div[2]/p[2]/span[3]').click()

In [4]:
allname=[] #存放全部名字
allpic=[]  #存放全部图片链接
allhref=[] #存放全部链接
baidu="https://www.baidu.com" #链接前缀

# 获取名字，图片链接，明星百科链接

In [5]:
while True:
    try:
        html=etree.HTML(driver.page_source,etree.HTMLParser())
        names=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[2]/a/text()')#获取这一页明星名字
        pictureurl=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[1]/a/img/@src')#获取这一页明星图片
        hrefs=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[1]/a/@href')#获取这一页明星链接
        for i in range(len(names)):
            allname.append(names[i])
            allpic.append(pictureurl[i])
            allhref.append(baidu+hrefs[i])
        next_btn=wait.until(EC.element_to_be_clickable(
            (By.XPATH,'//*[@id="1"]/div/div[1]/div[2]/div[2]/p/span[6]')))#等待下一页能被点击，没有出现下一页了退出
        next_btn.click()#点击下一页
        time.sleep(1)
    except:
        print('over')
        break
driver.close()

over


# 把信息汇总到data列表

In [7]:
data=[]
for i in range(len(allname)):
    data.append((allname[i],allpic[i],allhref[i]))

In [9]:
#保存为csv文件
f=open('明星.csv','w',newline='')
w=csv.writer(f)
for i in data:
    w.writerow(i)
f.close()

# 保存明星百科页面完整html源代码

In [9]:
head={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
htmls=[]
a=1
for i in data:
    name=i[0]
    url='https://baike.baidu.com/item/'+name
    r=requests.get(url,headers=head)
    r.encoding='utf-8'
    html=r.text
    htmls.append((url,html))
    print(a)
    a+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# 获取明星资料

In [43]:
stardata=[]
for html in htmls:
    try:
        nm=re.search(r'<dt class="basicInfo-item name">中文名</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">中文名</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        birth=re.search(r'<dt class="basicInfo-item name">出生日期</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">出生日期</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        #判断各种情况
        if birth!='':
            if '年' in birth and '月' in birth and '日' in birth:
                year,month,day=re.split('[年月日]',birth)[:3]
            elif '年' in birth and '月' in birth and '日' not in birth:
                year,month=re.split('[年月]',birth)[:2]
                day=''
            elif '年' in birth and '月' not in birth and '日' not in birth:
                year=re.split('[年]',birth)[0]
                month=''
                day=''
            elif '年' not in birth and '月' in birth and '日' in birth:
                year=''
                month,day=re.split('[月日]')[:2]
            elif '年' not in birth and '月' in birth and '日' not in birth:
                year=''
                month=re.split('[月]')[0]
                day=''
            elif '年' not in birth and '月' not in birth and '日' not in birth:
                year=''
                month=''
                day=''
            else:
                continue
        else:
            year=''
            month=''
            day=''
        height=re.search(r'<dt class="basicInfo-item name">身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        if ('CM' in height) or ('cm' in height):
            height=height[:-2]
        jiguan=re.search(r'<dt class="basicInfo-item name">出生地</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">出生地</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        xingzuo=re.search(r'<dt class="basicInfo-item name">星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        if nm=='':
            continue
        stardata.append((nm,'女',html[0],year,month,day,xingzuo,html[1],jiguan,height))
    except:
        print(nm+'获取失败')

王子文获取失败
王丽坤获取失败
毛晓彤获取失败
蓝心妍获取失败
张雯获取失败
王琦获取失败
获取失败
任娇获取失败
张蓝心获取失败
马秋梓（本名）获取失败
朱琳获取失败
陈思斯获取失败
刘敏获取失败
高洋获取失败
杨舒获取失败
孙茜获取失败
梁静获取失败
张龄心获取失败
林源获取失败
戴燕妮获取失败
路晨获取失败
甘露获取失败
万美汐获取失败
种丹妮获取失败
何彦霓获取失败
吴岱凝获取失败
涓子获取失败
战菁一获取失败
陈欣予获取失败
郑卫莉获取失败
刘芊含获取失败
李若嘉获取失败
文晓依获取失败
林静获取失败
刘孜获取失败
王霏霏获取失败
尤靖茹获取失败
程莉莎获取失败
刘庭羽获取失败
胡然获取失败
陈洁获取失败
钱琳琳获取失败
郑雅文获取失败
金玟岐获取失败
屈菁菁获取失败
娟子获取失败
涂凌获取失败
侯梦瑶获取失败
郑亦桐获取失败
董晴获取失败
李梦颖获取失败
岳秀清获取失败
金池获取失败
那笛获取失败
黄璐获取失败
张暖雅获取失败
王梓桐获取失败
叶迎春获取失败
倪睿思获取失败
白柳汐获取失败
毛泽少获取失败
方芳获取失败
瑛子获取失败
高姝瑶获取失败
何珈好获取失败
刘瑞琦获取失败
臧洪娜获取失败
陈苏获取失败
阿鲁阿卓获取失败
汪聪获取失败
刘晓晔获取失败
崔子格获取失败
韩烨获取失败
王若心获取失败
高叶获取失败
何佳怡获取失败
于越获取失败
石安妮获取失败
张墨锡获取失败
刘璐获取失败
肖雨雨获取失败
侃侃获取失败
夏一瑶获取失败
周显欣获取失败
武笑羽获取失败
王瑞子获取失败
张静静获取失败
毛琳获取失败
关昕获取失败
李佳璇获取失败
陈维涵获取失败
高晓菲获取失败
班嘉佳获取失败
刘娜萍获取失败
唐古获取失败
覃沐曦获取失败
白卉子获取失败
周庭伊获取失败
张倩获取失败
陈西贝获取失败
刘雨鑫获取失败
郭虹获取失败
邵夷贝获取失败
吴恙获取失败
血纯茗雅获取失败
范志博获取失败
李欣聪获取失败
林柯彤获取失败
衣珊获取失败
央金兰泽获取失败
缪婷茹获取失败
赵洁获取失败
李菲获取失败
张青获取失败
居文沛获取失败
夏力薪获取失败
顾璇获取失败
卫莱获取失败
赵悦童获取失败
张瑞竹获取失败
张一鸾获取失败
获取失败
获取失败
王玲玉获取失败
冷碗碗获取失败
张培获取失败
阎青妤获取失败
常春晓获取失败
张宇菲获取失败
骆文博获取失败
柴鸥获取失败


In [45]:
for i in stardata:
    print(i[0],i[3:6])

张檬 ('1988', '12', '29')
刘诗诗 ('1987', '3', '10')
周笔畅 ('1985', '7', '26')
张萌 ('1981', '3', '6')
杨幂 ('1986', '9', '12')
赵丽颖 ('1987', '10', '16')
唐嫣 ('1983', '12', '6')
迪丽热巴·迪力木拉提 ('1992', '6', '3')
杨颖 ('1989', '2', '28')
鞠婧祎 ('1994', '6', '18')
郑爽 ('1991', '8', '22')
杨紫 ('1992', '11', '6')
董璇 ('1979', '11', '5')
佟丽娅 ('1983', '8', '8')
刘雨昕 ('1997', '4', '20')
孙俪 ('1982', '9', '26')
宋茜 ('1987', '2', '2')
关晓彤 ('1997', '9', '17')
范冰冰 ('1981', '9', '16')
孙佳雨 ('1995', '10', '21')
吴倩 ('1992', '9', '26')
张子萱 ('1984', '2', '6')
李沁 ('1990', '9', '27')
唐艺昕 ('1987', '10', '9')
陈数 ('1977', '3', '9')
许佳琪 ('1995', '8', '27')
陈瑶 ('1994', '10', '31')
江疏影 ('1986', '9', '1')
赵今麦 ('2002', '9', '29')
李小璐 ('1981', '9', '30')
古力那扎尔·拜合提亚尔 ('1992', '5', '2')
刘涛 ('1978', '7', '12')
张馨予 ('1987', '3', '28')
张钧甯 ('1982', '9', '4')
邓紫棋 ('1991', '8', '16')
张天爱 ('', '', '')
潘之琳 ('1989', '9', '5')
柳岩 ('1980', '11', '8')
谭松韵 ('1990', '5', '31')
张佳宁 ('1989', '5', '26')
高圆圆 ('1979', '10', '5')
蒋欣 ('1983', '5', '8')
王菲 ('196

# 遍历打印

In [57]:
query = Star.select()
for s in query:
    print(s.name,s.year,s.month,s.day,s.height)
    a+=1

张檬 1988 12 29 165.0
刘诗诗 1987 3 10 165.0
周笔畅 1985 7 26 165.0
张萌 1981 3 6 169.0
杨幂 1986 9 12 166.5
赵丽颖 1987 10 16 165.0
唐嫣 1983 12 6 172.0
迪丽热巴·迪力木拉提 1992 6 3 168.0
鞠婧祎 1994 6 18 159.0
郑爽 1991 8 22 168.0
杨紫 1992 11 6 167.0
董璇 1979 11 5 168.0
佟丽娅 1983 8 8 164.5
孙俪 1982 9 26 165.0
宋茜 1987 2 2 168.0
关晓彤 1997 9 17 172.4
范冰冰 1981 9 16 168.0
吴倩 1992 9 26 166.0
张子萱 1984 2 6 171.0
李沁 1990 9 27 166.0
陈数 1977 3 9 168.0
许佳琪 1995 8 27 170.0
江疏影 1986 9 1 168.0
赵今麦 2002 9 29 165.0
李小璐 1981 9 30 162.5
古力那扎尔·拜合提亚尔 1992 5 2 172.0
张馨予 1987 3 28 169.0
张钧甯 1982 9 4 168.0
邓紫棋 1991 8 16 157.0
潘之琳 1989 9 5 167.0
柳岩 1980 11 8 162.0
谭松韵 1990 5 31 162.0
张佳宁 1989 5 26 168.0
高圆圆 1979 10 5 165.0
蒋欣 1983 5 8 171.0
王菲 1969 8 8 174.0
景甜 1988 7 21 167.0
赵薇 1976 3 12 166.0
韩雪 1983 1 11 170.0
高露 1982 10 8 167.0
戚薇 1984 10 26 168.0
金晨 1990 9 5 171.0
周冬雨 1992 1 31 162.0
王鸥 1982 10 28 170.0
宋佳 1980 11 13 173.0
白百何 1984 3 1 171.0
孙怡 1993 6 4 170.0
倪妮 1988 8 8 170.0
赵子琪 1976 6 17 165.0
谢娜 1981 5 6 166.0
马晓晨 1994 6 30 163.0
阚清子